# Homework

Question 1. Spark version
3.4.2

Question 2. FHV October 2019 partition size
6 MB

Question 3. Count records on 15th of October 
62610

Question 4. The longest trip
631152.5 hours

Question 5. Spark UI port
4040

Question 6. Least frequent pickup location zone
Jamaica Bay

In [ ]:
# Question 1 

In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/04 14:21:19 WARN Utils: Your hostname, farik-ThinkPad-E14-Gen-2 resolves to a loopback address: 127.0.1.1; using 192.168.8.105 instead (on interface wlp0s20f3)
24/03/04 14:21:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/04 14:21:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
pyspark.__version__

'3.4.2'

In [ ]:
# Question 2

In [5]:
import pandas as pd

In [8]:
df_fhv_pd = pd.read_csv('data/raw/fhv/2019/10/fhv_tripdata_2019-10.csv.gz', nrows=1000)

In [14]:
df_fhv_pd.dtypes

dispatching_base_num       object
pickup_datetime            object
dropOff_datetime           object
PUlocationID              float64
DOlocationID              float64
SR_Flag                   float64
Affiliated_base_number     object
dtype: object

In [15]:
spark.createDataFrame(df_fhv_pd).schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', DoubleType(), True), StructField('DOlocationID', DoubleType(), True), StructField('SR_Flag', DoubleType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [16]:
from pyspark.sql import types

In [17]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.StringType(), True), 
    types.StructField('dropOff_datetime', types.StringType(), True), 
    types.StructField('PUlocationID', types.IntegerType(), True), 
    types.StructField('DOlocationID', types.IntegerType(), True), 
    types.StructField('SR_Flag', types.StringType(), True), 
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [18]:
df_fhv = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('data/raw/fhv/2019/10/fhv_tripdata_2019-10.csv.gz')

In [19]:
df_fhv \
    .repartition(6) \
    .write.parquet('data/pq/fhv/2019/10/')

In [20]:
!ls -lhR data/pq/fhv/2019

data/pq/fhv/2019:
total 4,0K
drwxr-xr-x 2 farik farik 4,0K Mar  4 14:36 10

data/pq/fhv/2019/10:
total 39M
-rw-r--r-- 1 farik farik 6,5M Mar  4 14:36 part-00000-b74f94eb-2d12-4707-afe2-0bacdf7138d0-c000.snappy.parquet
-rw-r--r-- 1 farik farik 6,5M Mar  4 14:36 part-00001-b74f94eb-2d12-4707-afe2-0bacdf7138d0-c000.snappy.parquet
-rw-r--r-- 1 farik farik 6,5M Mar  4 14:36 part-00002-b74f94eb-2d12-4707-afe2-0bacdf7138d0-c000.snappy.parquet
-rw-r--r-- 1 farik farik 6,5M Mar  4 14:36 part-00003-b74f94eb-2d12-4707-afe2-0bacdf7138d0-c000.snappy.parquet
-rw-r--r-- 1 farik farik 6,5M Mar  4 14:36 part-00004-b74f94eb-2d12-4707-afe2-0bacdf7138d0-c000.snappy.parquet
-rw-r--r-- 1 farik farik 6,5M Mar  4 14:36 part-00005-b74f94eb-2d12-4707-afe2-0bacdf7138d0-c000.snappy.parquet
-rw-r--r-- 1 farik farik    0 Mar  4 14:36 _SUCCESS


In [ ]:
# question 3

In [23]:
df_trips_data = spark.read.parquet('data/pq/fhv/2019/10/')

In [32]:
df_trips_data.createOrReplaceTempView('trips_data')

In [80]:
df_result = spark.sql("""
SELECT 
    COUNT(1) AS count
FROM
    trips_data
WHERE
    day(pickup_datetime) = 15 AND month(pickup_datetime) = 10
""")


In [81]:
df_result.show()

+-----+
|count|
+-----+
|62610|
+-----+



In [ ]:
# Question 4

In [44]:
df_result = spark.sql("""
SELECT 
    MAX((unix_timestamp(dropOff_datetime) - unix_timestamp(pickup_datetime)) / 3600) AS max_diff
FROM
    trips_data
""")

In [45]:
df_result.show()

+--------+
|max_diff|
+--------+
|631152.5|
+--------+



In [ ]:
# Question 6

In [46]:
df_zones = spark.read.parquet('zones/')

In [55]:
df_join = df_trips_data.join(df_zones, df_trips_data.PUlocationID == df_zones.LocationID)

In [59]:
df_join.createOrReplaceTempView('trips_join')

In [68]:
df_result = spark.sql("""
SELECT 
    COUNT(1) AS count,
    Zone
FROM
    trips_join
GROUP BY 
    Zone
ORDER BY 
    1 ASC
""")


In [69]:
df_result.show()

+-----+--------------------+
|count|                Zone|
+-----+--------------------+
|    1|         Jamaica Bay|
|    2|Governor's Island...|
|    5| Green-Wood Cemetery|
|    8|       Broad Channel|
|   14|     Highbridge Park|
|   15|        Battery Park|
|   23|Saint Michaels Ce...|
|   25|Breezy Point/Fort...|
|   26|Marine Park/Floyd...|
|   29|        Astoria Park|
|   39|    Inwood Hill Park|
|   47|       Willets Point|
|   53|Forest Park/Highl...|
|   57|  Brooklyn Navy Yard|
|   62|        Crotona Park|
|   77|        Country Club|
|   89|     Freshkills Park|
|   98|       Prospect Park|
|  105|     Columbia Street|
|  110|  South Williamsburg|
+-----+--------------------+
only showing top 20 rows

